# Importing important modules and setting up the project

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

import xgboost

from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import tensorflow as tf
from keras import backend as K

from sklearn.model_selection import GridSearchCV

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/my-sample-data/test.csv
/kaggle/input/my-sample-data/train.csv
/kaggle/input/fare-classification/meta_data.csv


Using TensorFlow backend.


Reading the data from files

In [2]:
train = pd.read_csv('../input/my-sample-data/train.csv')
test = pd.read_csv('../input/my-sample-data/test.csv')

In [3]:
train.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct


In [4]:
test.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare
0,213284604,10.5,924,42,2.4486,148,2/1/2020 0:38,2/1/2020 0:53,6.83454,79.8750,6.77490,79.8840,289.27
1,213286352,10.5,4249,20,0.0000,91,2/1/2020 1:02,2/1/2020 2:13,6.91168,79.8723,6.55091,79.9706,1912.70
2,213293973,10.5,1552,255,2.6588,23,2/1/2020 5:02,2/1/2020 5:28,6.92145,79.8478,6.90539,79.8989,394.00
3,213294622,10.5,462,16,0.0000,198,2/1/2020 5:30,2/1/2020 5:38,6.77433,79.9416,6.80401,79.9407,154.32
4,213298687,10.5,814,392,12.3692,69,2/1/2020 7:00,2/1/2020 7:14,6.97968,79.9130,6.98875,79.8914,147.47


In [22]:
predictor_cols = ['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare', 'meter_waiting_till_pickup', 'fare']
train_X = train[predictor_cols]
test_X = test[predictor_cols]

#train_y.fillna(train_y.mean(), inplace=True)
train.fillna(train_X.mean(), inplace=True)
test.fillna(test_X.mean(), inplace=True)
train['num_label'] = False
train['direct_distance'] = 0.0
test['direct_distance'] = 0.0

train['pickup_time'] = pd.to_datetime(train['pickup_time'], errors='coerce')
test['pickup_time'] = pd.to_datetime(test['pickup_time'], errors='coerce')

train['drop_time'] = pd.to_datetime(train['drop_time'], errors='coerce')
test['drop_time'] = pd.to_datetime(test['drop_time'], errors='coerce')

train['pick_hour'] = train['pickup_time'].dt.hour
train['drop_hour'] = train['drop_time'].dt.hour

test['pick_hour'] = test['pickup_time'].dt.hour
test['drop_hour'] = test['drop_time'].dt.hour


for index, row in train.iterrows():
    lat1, lng1, lat2, lng2 = map(np.radians, (row['drop_lat'], row['drop_lon'], row['pick_lat'], row['pick_lon']))
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * 6371 * np.arcsin(np.sqrt(d))
    train.at[index,'direct_distance'] = 2 * 6371 * np.arcsin(np.sqrt(d))
    #d = (((row['drop_lon'] - row['pick_lon'])**2)+((row['drop_lat'] - row['pick_lat'])**2))
    #train.at[index,'direct_distance'] = 2 * 6371 * np.arcsin()
    
for index, row in test.iterrows():
    lat1, lng1, lat2, lng2 = map(np.radians, (row['drop_lat'], row['drop_lon'], row['pick_lat'], row['pick_lon']))
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * 6371 * np.arcsin(np.sqrt(d))
    test.at[index,'direct_distance'] = 2 * 6371 * np.arcsin(np.sqrt(d))


train['fare_for_duration'] = (train['fare'] - train['meter_waiting_fare'])/train['duration']
#train['fare_for_distance'] = (train['fare'] - train['meter_waiting_fare'])/train['direct_distance']
train['fare_for_time'] = (train['fare'] - train['meter_waiting_fare'])/(train['duration']+train['meter_waiting_till_pickup'])
#train['distance_for_time'] = train['direct_distance']/train['duration']
train['meter_waiting_to_duration'] = train['meter_waiting']/train['duration']
train['additional_fare_to_duration'] = train['additional_fare']/train['duration']
#train['additional_fare_to_distance'] = train['additional_fare']/train['direct_distance']
train['additional_fare_to_fare'] = train['additional_fare']/(train['fare']+train['additional_fare'])
train['mtr_wating_fare_to_waiting_duration'] = train['meter_waiting_fare']/(train['meter_waiting']+train['meter_waiting_till_pickup'])

test['fare_for_duration'] = (test['fare'] - test['meter_waiting_fare'])/test['duration']
#test['fare_for_distance'] = (test['fare'] - test['meter_waiting_fare'])/test['direct_distance']
test['fare_for_time'] = (test['fare'] - test['meter_waiting_fare'])/(test['duration']+test['meter_waiting_till_pickup'])
#test['distance_for_time'] = test['direct_distance']/test['duration']
test['meter_waiting_to_duration'] = test['meter_waiting']/test['duration']
test['additional_fare_to_duration'] = test['additional_fare']/test['duration']
#test['additional_fare_to_distance'] = test['additional_fare']/test['direct_distance']
test['additional_fare_to_fare'] = test['additional_fare']/(test['fare']+test['additional_fare'])
test['mtr_wating_fare_to_waiting_duration'] = test['meter_waiting_fare']/(test['meter_waiting']+test['meter_waiting_till_pickup'])

train.replace([np.inf, -np.inf], np.nan)
test.replace([np.inf, -np.inf], np.nan)
train.fillna(train.mean(), inplace=True)
test.fillna(test.mean(), inplace=True)

train['fare_for_duration'] = train['fare_for_duration'].astype(np.float32)
#train['fare_for_distance'] = train['fare_for_distance'].astype(np.float32)
train['fare_for_time'] = train['fare_for_time'].astype(np.float32)
#train['distance_for_time'] = train['distance_for_time'].astype(np.float32)
train['meter_waiting_to_duration'] = train['meter_waiting_to_duration'].astype(np.float32)
train['additional_fare_to_duration'] = train['additional_fare_to_duration'].astype(np.float32)
#train['additional_fare_to_distance'] = train['additional_fare_to_distance'].astype(np.float32)
train['additional_fare_to_fare'] = train['additional_fare_to_fare'].astype(np.float32)
train['mtr_wating_fare_to_waiting_duration'] = train['mtr_wating_fare_to_waiting_duration'].astype(np.float32)

test['fare_for_duration'] = test['fare_for_duration'].astype(np.float32)
#test['fare_for_distance'] = test['fare_for_distance'].astype(np.float32)
test['fare_for_time'] = test['fare_for_time'].astype(np.float32)
#test['distance_for_time'] = test['distance_for_time'].astype(np.float32)
test['meter_waiting_to_duration'] = test['meter_waiting_to_duration'].astype(np.float32)
test['additional_fare_to_duration'] = test['additional_fare_to_duration'].astype(np.float32)
#test['additional_fare_to_distance'] = test['additional_fare_to_distance'].astype(np.float32)
test['additional_fare_to_fare'] = test['additional_fare_to_fare'].astype(np.float32)
test['mtr_wating_fare_to_waiting_duration'] = test['mtr_wating_fare_to_waiting_duration'].astype(np.float32)



for index, row in train.iterrows():
    if(row['label'] == "correct"):
        train.at[index, 'num_label'] = 1
    else:
        train.at[index, 'num_label'] = 0


#train_y = train['label']

In [23]:
# 'fare_for_distance',
# 'distance_for_time', 'additional_fare_to_distance', 'distance_for_time', 'only_fare_for_duration',
predictor_cols = ['additional_fare', 'mtr_wating_fare_to_waiting_duration', 'fare_for_time', 'meter_waiting_to_duration', 'fare_for_duration', 'additional_fare_to_duration', 'additional_fare_to_fare', 'pick_hour', 'drop_hour', 'pick_lat', 'pick_lon', 'direct_distance', 'duration', 'meter_waiting', 'meter_waiting_fare', 'meter_waiting_till_pickup', 'fare']
train_X_ = train[predictor_cols]
test_X = test[predictor_cols]
train_X_.replace([np.inf, -np.inf], np.nan)
test_X.replace([np.inf, -np.inf], np.nan)
train_X_.fillna(train_X_.mean(), inplace=True)
test_X.fillna(test_X.mean(), inplace=True)

train_y_ = train.num_label

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [24]:
train_X_

,additional_fare,fare_for_distance,mtr_wating_fare_to_waiting_duration,fare_for_time,meter_waiting_to_duration,fare_for_duration,additional_fare_to_duration,additional_fare_to_fare,pick_hour,drop_hour,pick_lat,pick_lon,direct_distance,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare
0,10.500000,53.079170,0.000000,0.301024,0.067146,0.324125,0.012590,0.037391,0,0,6.86252,79.8993,5.092770,834.000000,56.000000,0.000000,64.000000,270.32
1,10.500000,62.451515,0.000000,0.213892,0.059418,0.250126,0.013274,0.050396,0,1,6.88589,79.8984,3.168058,791.000000,47.000000,0.000000,134.000000,197.85
2,10.500000,47.838398,0.000000,0.262753,0.073597,0.277498,0.009660,0.033639,1,1,6.90839,79.8651,6.305395,1087.000000,80.000000,0.000000,61.000000,301.64
3,10.500000,77.309006,0.046206,0.100054,0.453177,0.111432,0.017559,0.113147,2,2,6.92570,79.8895,0.861946,598.000000,271.000000,15.663800,68.000000,82.30
4,13.719651,40.051678,0.043231,0.179765,0.369423,0.191638,0.008057,0.036870,3,3,6.87441,79.8615,8.147782,1702.858077,629.074231,32.057666,112.466832,358.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,10.500000,91.593201,0.009967,0.149603,0.110979,0.230117,0.012530,0.050297,22,22,7.29073,80.6367,2.105376,838.000000,93.000000,5.421900,451.000000,198.26
17172,10.500000,53.479004,0.000000,0.265402,0.198977,0.270214,0.004881,0.017745,23,23,6.90569,79.8516,10.868377,2151.000000,428.000000,0.000000,39.000000,581.23
17173,10.500000,72.882500,0.000000,0.204290,0.034221,0.289734,0.039924,0.121107,23,23,7.09210,79.9000,1.045518,263.000000,9.000000,0.000000,110.000000,76.20
17174,10.500000,46.303036,0.000000,0.113455,0.134033,0.155373,0.012238,0.073013,23,23,6.94540,79.8768,2.879077,858.000000,115.000000,0.000000,317.000000,133.31


In [9]:
test_X

,additional_fare,mtr_wating_fare_to_waiting_duration,fare_for_time,meter_waiting_to_duration,fare_for_duration,additional_fare_to_duration,additional_fare_to_fare,pick_hour,drop_hour,pick_lat,pick_lon,direct_distance,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare
0,10.5,0.012887,0.267557,0.045455,0.310413,0.011364,0.035027,0,0,6.83454,79.8750,6.705702,924,42,2.44860,148,289.27
1,10.5,0.000000,0.440714,0.004707,0.450153,0.002471,0.005460,1,2,6.91168,79.8723,41.558513,4249,20,0.00000,91,1912.70
2,10.5,0.009564,0.248471,0.164304,0.252153,0.006765,0.025958,5,5,6.92145,79.8478,5.916678,1552,255,2.65880,23,394.00
3,10.5,0.000000,0.233818,0.034632,0.334026,0.022727,0.063706,5,5,6.77433,79.9416,3.301761,462,16,0.00000,198,154.32
4,10.5,0.026831,0.153002,0.481572,0.165972,0.012899,0.066468,7,7,6.97968,79.9130,2.588542,814,392,12.36920,69,147.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8571,10.5,0.057485,0.210688,0.248984,0.211054,0.006094,0.026317,21,21,6.85103,79.9567,3.934272,1723,429,24.83332,3,388.48
8572,10.5,0.000000,0.252728,0.058055,0.275653,0.007620,0.026899,21,22,6.91293,79.9656,7.517433,1378,80,0.00000,125,379.85
8573,10.5,0.022043,0.214297,0.133971,0.261975,0.025120,0.085165,22,22,6.85718,79.9081,2.057225,418,56,3.28440,93,112.79
8574,10.5,0.056061,0.133736,0.341646,0.135153,0.006546,0.040547,22,22,6.91289,79.8846,3.900888,1604,548,31.67440,17,248.46


In [25]:
train_X, val_X, train_y, val_y = train_test_split(train_X_, train_y_, test_size = 0.2, train_size = 0.8, random_state = 0)

In [11]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    mse = mean_squared_error(test_labels, predictions)
    mae = mean_absolute_error(test_labels, predictions)
    rmse = np.sqrt(mse)
    score=cross_val_score(model, test_features, test_labels, cv=10)
    accuracy = accuracy_score(test_labels, predictions.round())
    print('Model Performance')
    print('Accuracy:%f'%accuracy)
    print("Mean cross validation score:%f"%score.mean())
    print('Mean Squared Error : %.4f' % mse)
    print('Root MSE : %.4f' % rmse)

Random forest model

In [13]:
my_random_forest_model = RandomForestRegressor(n_estimators=100, random_state = 0)
my_random_forest_model.fit(train_X, train_y)

evaluate(my_random_forest_model, val_X, val_y)

pred_classes = my_random_forest_model.predict(val_X)
predi = []
for x in range(len(pred_classes)):
    predi.append(bool(pred_classes[x]))
    
pred_classes = np.array(predi)

f1 = f1_score(val_y, pred_classes)
print('F1 score: %f' % f1)
# Multiply by -1 since sklearn calculates *negative* MAE
#scores = cross_val_score(my_pipeline, X, y, cv=5, scoring='neg_mean_absolute_error')
#scores = cross_val_score(my_random_forest_model, train_X, train_y.round(), scoring='f1')
#print("MAE scores:\n", scores)


Model Performance
Accuracy:0.957218
Mean cross validation score:0.504798
Mean Squared Error : 0.0364
Root MSE : 0.1909
F1 score: 0.958141


Decision tree model

In [26]:
my_decision_tree_model = DecisionTreeRegressor(min_samples_split=100,
        max_features="auto", random_state=50, 
        max_depth=100)
my_decision_tree_model.fit(train_X, train_y)

evaluate(my_decision_tree_model, val_X, val_y)

pred_classes = my_decision_tree_model.predict(val_X)
predi = []
for x in range(len(pred_classes)):
    predi.append(bool(pred_classes[x]))
    
pred_classes = np.array(predi)

f1 = f1_score(val_y, pred_classes)
print('F1 score: %f' % f1)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

ADA Boost model

In [14]:
Ada_boost_model = AdaBoostRegressor()
Ada_boost_model.fit(train_X, train_y)

evaluate(Ada_boost_model, val_X, val_y)

pred_classes = Ada_boost_model.predict(val_X)
predi = []
for x in range(len(pred_classes)):
    predi.append(bool(pred_classes[x]))
    
pred_classes = np.array(predi)

f1 = f1_score(val_y, pred_classes)
print('F1 score: %f' % f1)

Model Performance
Accuracy:0.939464
Mean cross validation score:0.186050
Mean Squared Error : 0.0826
Root MSE : 0.2874
F1 score: 0.951000


In [15]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(C=1, penalty='l1', solver='liblinear')
param_grid = {'C':[0.001,.009,0.01,.09,1,5,10,25]}
'''{
    'n_estimators' : [10, 200],
    'max_features' : ['auto', 'sqrt', 'log2', 0.5]
}'''
gs_model = GridSearchCV(clf, param_grid = param_grid, scoring = 'recall')
gs_model.fit(train_X, train_y)

predictions = gs_model.predict(val_X)
accuracy = accuracy_score(val_y, predictions.round())
print('Accuracy:%f'%accuracy)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

Accuracy:0.911816


In [16]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
#def create_baseline():
'''
model = Sequential()
model.add(Dense(7, activation='relu'))
model.add(Dense(7, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(7, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(7, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(7, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(7, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(7, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(7, activation='relu'))
#model.add(Dropout(0.4))
model.add(Dense(1, activation='softmax'))
#model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.compile(loss='mse', optimizer='Adam', metrics=[tf.keras.metrics.MeanSquaredError(),tf.keras.metrics.Accuracy()])
#tf.keras.metrics.BinaryAccuracy()
#adam,sgd
#return model
'''
from keras.layers import BatchNormalization
model = Sequential()
model.add(Dense(15))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(32,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(16,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(8,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1,activation='softmax'))

model.compile(optimizer='adam',loss='mse', metrics=['accuracy',f1_m])
#model.compile(loss=keras.losses.categorical_crossentropy,optimizer='adam',metrics=['accuracy'])

#estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=10, verbose=0)
#kfold = StratifiedKFold(n_splits=10, shuffle=True)
#results = cross_val_score(estimator, train_X, train_y, cv=kfold)
#print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
model.fit(train_X_.values, train_y_.values,
          batch_size=20,
          epochs=5,
          validation_split = 0.2,
          verbose = 1,
          shuffle=True)

#evaluate(model, val_X, val_y)
pred_classes = model.predict(val_X)
predi = []
for x in range(len(pred_classes)):
    predi.append(bool(pred_classes[x]))
    
pred_classes = np.array(predi)

f1 = f1_score(val_y, pred_classes)
print('F1 score: %f' % f1)

Train on 13740 samples, validate on 3436 samples
Epoch 1/5
13740/13740 [==============================] - 8s 598us/step - loss: 0.0999 - accuracy: 0.9001 - f1_m: 0.9461 - val_loss: 0.0896 - val_accuracy: 0.9104 - val_f1_m: 0.9517
Epoch 2/5
13740/13740 [==============================] - 5s 370us/step - loss: 0.0999 - accuracy: 0.9001 - f1_m: 0.9459 - val_loss: 0.0896 - val_accuracy: 0.9104 - val_f1_m: 0.9517
Epoch 3/5
13740/13740 [==============================] - 5s 369us/step - loss: 0.0999 - accuracy: 0.9001 - f1_m: 0.9460 - val_loss: 0.0896 - val_accuracy: 0.9104 - val_f1_m: 0.9517
Epoch 4/5
13740/13740 [==============================] - 5s 386us/step - loss: 0.0999 - accuracy: 0.9001 - f1_m: 0.9462 - val_loss: 0.0896 - val_accuracy: 0.9104 - val_f1_m: 0.9517
Epoch 5/5
13740/13740 [==============================] - 5s 382us/step - loss: 0.0999 - accuracy: 0.9001 - f1_m: 0.9461 - val_loss: 0.0896 - val_accuracy: 0.9104 - val_f1_m: 0.9517
F1 score: 0.951000


In [18]:
xg_model = xgboost.XGBClassifier(base_score=0.1, booster= None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0.4, gpu_id=0, importance_type='gain',
              interaction_constraints=None, learning_rate=0.01,
              max_delta_step=0, max_depth=80, min_child_weight=7, missing=None,
              monotone_constraints=None, n_estimators=1000, n_jobs=6, nthread=6,
              num_parallel_tree=1, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=False,
              subsample=0.8,tree_method='hist', validate_parameters=False,
              verbosity=1)
#xg_model = xgboost.XGBClassifier()
#base_score 0.5 -> 0.1
#score=cross_val_score(xg_model, train_X, train_y, cv=10)
xg_model.fit(train_X, train_y)


XGBClassifier(base_score=0.1, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0.4, gpu_id=0, importance_type='gain',
              interaction_constraints=None, learning_rate=0.01,
              max_delta_step=0, max_depth=80, min_child_weight=7, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=6, nthread=6,
              num_parallel_tree=1, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=False,
              subsample=0.8, tree_method='hist', validate_parameters=False,
              verbosity=1)

In [19]:
def xg_evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    #mse = mean_squared_error(test_labels, predictions)
    #mae = mean_absolute_error(test_labels, predictions)
    #rmse = np.sqrt(mse)
    #score=cross_val_score(model, test_features, test_labels, cv=10)
    accuracy = accuracy_score(test_labels, predictions.round())
    #print('Model Performance')
    print('Accuracy:%f'%accuracy)
    #print("Mean cross validation score:%f"%score.mean())
    #print('Mean Squared Error : %.4f' % mse)
    #print('Root MSE : %.4f' % rmse)
xg_evaluate(xg_model, val_X, val_y)
pred_classes = xg_model.predict(val_X)
predi = []
for x in range(len(pred_classes)):
    predi.append(bool(pred_classes[x]))
    
pred_classes = np.array(predi)

f1 = f1_score(val_y, pred_classes)
print('F1 score: %f' % f1)

Accuracy:0.956054
F1 score: 0.975982


In [20]:
from sklearn.neighbors import KNeighborsClassifier
knn_classifier = KNeighborsClassifier(n_neighbors=7)
knn_classifier.fit(train_X, train_y)

def knn_evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    #mse = mean_squared_error(test_labels, predictions)
    #mae = mean_absolute_error(test_labels, predictions)
    #rmse = np.sqrt(mse)
    #score=cross_val_score(model, test_features, test_labels, cv=10)
    accuracy = accuracy_score(test_labels, predictions.round())
    #print('Model Performance')
    print('Accuracy:%f'%accuracy)
    #print("Mean cross validation score:%f"%score.mean())
    #print('Mean Squared Error : %.4f' % mse)
    #print('Root MSE : %.4f' % rmse)
knn_evaluate(knn_classifier, val_X, val_y)

pred_classes = knn_classifier.predict(val_X)
predi = []
for x in range(len(pred_classes)):
    predi.append(bool(pred_classes[x]))
    
pred_classes = np.array(predi)

f1 = f1_score(val_y, pred_classes)
print('F1 score: %f' % f1)

Accuracy:0.946740
F1 score: 0.971140


In [21]:
predicted_rf = my_random_forest_model.predict(test_X)
predicted_dt = my_decision_tree_model.predict(test_X)
predicted_ad = Ada_boost_model.predict(test_X)
predicted_xg = xg_model.predict(test_X)
predicted_NN = model.predict(test_X)
predicted_KNN = knn_classifier.predict(test_X)

pred_KNN = []
for x in range(len(predicted_KNN)):
    pred_KNN.append(bool(predicted_KNN[x]))

predicted_KNN = np.array(pred_KNN)

pred_NN = []
for x in range(len(predicted_NN)):
    pred_NN.append(bool(predicted_NN[x]))

predicted_NN = np.array(pred_NN)
    
pred_rf = []
for x in range(len(predicted_rf)):
    pred_rf.append(bool(predicted_rf[x]))
    
predicted_rf = np.array(pred_rf)
    
pred_dt = []
for x in range(len(predicted_dt)):
    pred_dt.append(bool(predicted_dt[x]))

predicted_dt = np.array(pred_dt)

pred_ad = []
for x in range(len(predicted_ad)):
    pred_ad.append(bool(predicted_ad[x]))

predicted_ad = np.array(pred_ad)

pred_xg = []
for x in range(len(predicted_xg)):
    pred_xg.append(bool(predicted_xg[x]))
    
predicted_xg = np.array(pred_xg)

output_rf = pd.DataFrame({'tripid': test.tripid,
                       'prediction': predicted_rf})

output_dt = pd.DataFrame({'tripid': test.tripid,
                       'prediction': predicted_dt})

output_adb = pd.DataFrame({'tripid': test.tripid,
                       'prediction': predicted_dt})

output_xg = pd.DataFrame({'tripid': test.tripid,
                       'prediction': predicted_xg})

output_NN = pd.DataFrame({'tripid': test.tripid,
                       'prediction': predicted_NN})

output_KNN = pd.DataFrame({'tripid': test.tripid,
                       'prediction': predicted_KNN})

output_rf_path = "submission_rf.csv"
output_dt_path = "submission_dt.csv"
output_adb_path = "submission_adb.csv"
output_xg_path = "submission_xg.csv"
output_NN_path = "submission_nn.csv"
output_KNN_path = "submission_knn.csv"

output_rf.to_csv(output_rf_path, index=False)
output_dt.to_csv(output_dt_path, index=False)
output_adb.to_csv(output_adb_path, index = False)
output_xg.to_csv(output_xg_path, index = False)
output_NN.to_csv(output_NN_path, index = False)
output_KNN.to_csv(output_KNN_path, index = False)